In [2]:
import os
import numpy as np
import pandas as pd
import boto3
from mtcnn.mtcnn import MTCNN
import PIL
from PIL import Image, ImageOps
from numpy import asarray
from os import listdir
from os.path import isfile, join
import cv2
import requests
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
import random
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
%matplotlib inline

## Read in CSV with test data information to begin facial detection

In [3]:
face_data = pd.read_csv('updated_test_data.csv')
face_data

,images,mask_on,no_mask,num_people_image,pred_num_people,corr_pred,detect_type
0,mask-dataset/images/test/2815.png,0,1,1,1,1,MTCNN
1,mask-dataset/images/test/6436.jpg,1,0,1,1,1,MTCNN
2,mask-dataset/images/test/4035.jpg,1,1,2,2,1,MTCNN
3,mask-dataset/images/test/5848.jpg,3,0,3,3,1,MTCNN
4,mask-dataset/images/test/5603.jpg,2,0,2,2,1,Haarcascade
...,...,...,...,...,...,...,...
245,mask-dataset/images/test/3067.jpg,0,1,1,1,1,MTCNN
246,mask-dataset/images/test/3805.jpg,1,0,1,1,1,MTCNN
247,mask-dataset/images/test/4051.jpg,1,0,1,2,0,Haarcascade
248,mask-dataset/images/test/662.png,1,0,1,1,1,MTCNN


In [7]:
face_data['pred_mask_on'] = np.nan
face_data['pred_no_mask'] = np.nan

In [8]:
face_data.head(10)

,images,mask_on,no_mask,num_people_image,pred_num_people,corr_pred,detect_type,pred_mask_on,pred_no_mask
0,mask-dataset/images/test/2815.png,0,1,1,1,1,MTCNN,NaN,NaN
1,mask-dataset/images/test/6436.jpg,1,0,1,1,1,MTCNN,NaN,NaN
2,mask-dataset/images/test/4035.jpg,1,1,2,2,1,MTCNN,NaN,NaN
3,mask-dataset/images/test/5848.jpg,3,0,3,3,1,MTCNN,NaN,NaN
4,mask-dataset/images/test/5603.jpg,2,0,2,2,1,Haarcascade,NaN,NaN
5,mask-dataset/images/test/187.jpg,5,0,5,4,0,Haarcascade,NaN,NaN
6,mask-dataset/images/test/155.jpg,6,1,7,9,0,MTCNN,NaN,NaN
7,mask-dataset/images/test/2175.jpg,1,0,1,3,0,Haarcascade,NaN,NaN
8,mask-dataset/images/test/5065.jpg,3,0,3,3,1,MTCNN,NaN,NaN
9,mask-dataset/images/test/2652.png,1,0,1,1,1,MTCNN,NaN,NaN


# Create Mask detection model

## USING MTCNN AND HAARCASCADE

### IMPROPER MASK ADDED TO PROPER MASK

In [9]:
pred_mask_count = {}
pred_no_mask_count = {}

for i in range(250):
    pic = face_data.iloc[i]['images']
    detect = face_data.iloc[i]['detect_type']
    im = Image.open(os.path.abspath(
        os.path.abspath(os.getcwd())+'/'+pic), mode='r')
    if im.mode != 'RGB':
        pixels = np.array(Image.open(os.path.abspath(
            os.getcwd())+'/'+pic, mode='r').convert('RGB'))
    elif im.mode == 'RGB':
        pixels = np.array(Image.open(
            os.path.abspath(os.getcwd())+'/'+pic, mode='r'))
    gray_image = cv2.cvtColor(pixels, cv2.COLOR_BGR2GRAY)

    im = cv2.imread(os.path.abspath(os.getcwd())+'/'+pic)
    
    #Haarcascades
    face_cascade = cv2.CascadeClassifier(os.path.abspath(os.getcwd())+'/haarcascade/haarcascade_frontalface_default.xml')
    profile_cascade = cv2.CascadeClassifier(os.path.abspath(os.getcwd())+'/haarcascade/haarcascade_profileface.xml')
    nose_cascade = cv2.CascadeClassifier(os.path.abspath(os.getcwd())+'/haarcascade/haarcascade_nose.xml')
    mouth_cascade = cv2.CascadeClassifier(os.path.abspath(os.getcwd())+'/haarcascade/haarcascade_mouth.xml')
    
    if detect == 'MTCNN':
        print(i, ' Using MTCNN')
        detector = MTCNN()

        # detect faces in the image
        faces = detector.detect_faces(pixels)

        # mask wear
        proper_mask_count = 0
#         improper_mask_count = 0
        no_mask_count = 0
        for result in faces:
            if result['confidence'] >= 0.75:
                x, y, w, h = result['box']
                cv2.rectangle(pixels, (x, y), (x+w, y+h), (255, 0, 0), 2)
                
                roi_image = gray_image[y:y+h, x:x+w]
                roi_color = pixels[y:y+h, x:x+w]
#                 plt.imshow(roi_color)
#                 ax = plt.gca()
                
#                 rect = Rectangle((x, y), w, h, fill=False, color='red')
# #             # draw the box
#                 ax.add_patch(rect)
                
                noses = nose_cascade.detectMultiScale(roi_image, scaleFactor=1.1, minNeighbors=20)
                for (nx,ny,nw,nh) in noses: 
                    cv2.rectangle(roi_image,(nx,ny),(nx+nw,ny+nh),(0,255,0),2)
#                     no = Rectangle((nx, ny), nw, nh, fill=False, color='green')
#                     ax.add_patch(no)
                mouth = mouth_cascade.detectMultiScale(roi_image, scaleFactor=1.3, minNeighbors=40)
                for (mx,my,mw,mh) in mouth: 
                    cv2.rectangle(roi_image,(mx,my),(mx+mw,my+mh),(0,0,255),2)
#                     mo = Rectangle((mx, my), mw, mh, fill=False, color='blue')
#                     ax.add_patch(mo)
#                 plt.show()

                num_nose = len(noses)
                num_mouth = len(mouth)
#                 print(f'Noses: {num_nose}, Mouths: {num_mouth}')
                
                if num_nose == 0 and num_mouth == 0: # Proper masking
                    proper_mask_count += 1
                elif num_nose != 0 and num_mouth == 0: # Improper masking
                    proper_mask_count += 1
                elif num_nose != 0 and num_mouth != 0: # no masking
                    no_mask_count += 1
        print(f"Proper Mask: {proper_mask_count}, No Mask: {no_mask_count}")
                        
    if detect == "Haarcascade":
        print(i, ' Using Haarcascade')
        proper_mask_count = 0
#         improper_mask_count = 0
        no_mask_count = 0
        f = []
        detected_faces = face_cascade.detectMultiScale(image=im, scaleFactor=1.05, minNeighbors=8)
        detected_prof = profile_cascade.detectMultiScale(image=im, scaleFactor=1.05, minNeighbors=4)
        f.append(detected_faces)
        f.append(detected_prof)
        faces = [x for x in f if len(x) != 0]
        for face in faces:
            for (x, y, w, h) in face:
#                 print(x,y,w,h)
#                 cv2.rectangle(pixels, (x, y), (x+w, y+h), (255, 0, 0), 2)

                roi_image = gray_image[y:y+h, x:x+w]
                roi_color = pixels[y:y+h, x:x+w]
#                 plt.imshow(roi_color)
#                 ax = plt.gca()

#                 rect = Rectangle((x, y), w, h, fill=False, color='red')
#     #             # draw the box
#                 ax.add_patch(rect)

                noses = nose_cascade.detectMultiScale(roi_image, scaleFactor=1.1, minNeighbors=20)
                for (nx,ny,nw,nh) in noses: 
                    cv2.rectangle(roi_image,(nx,ny),(nx+nw,ny+nh),(0,255,0),2)
#                     no = Rectangle((nx, ny), nw, nh, fill=False, color='green')
#                     ax.add_patch(no)
                
                mouth = mouth_cascade.detectMultiScale(roi_image, scaleFactor=1.3, minNeighbors=40)
                for (mx,my,mw,mh) in mouth: 
                    cv2.rectangle(roi_image,(mx,my),(mx+mw,my+mh),(0,0,255),2)
#                     mo = Rectangle((mx, my), mw, mh, fill=False, color='blue')
#                     ax.add_patch(mo)
#                 plt.show()

            num_nose = len(noses)
            num_mouth = len(mouth)
#             print(f'Noses: {num_nose}, Mouths: {num_mouth}')

            if num_nose == 0 and num_mouth == 0: # Proper masking
                proper_mask_count += 1
            elif num_nose != 0 and num_mouth == 0: # Improper masking
                proper_mask_count += 1
            elif num_nose != 0 and num_mouth != 0: # no masking
                no_mask_count += 1
        print(f"Proper Mask: {proper_mask_count}, No Mask: {no_mask_count}")

    pred_mask_count[i] = proper_mask_count
    pred_no_mask_count[i] = no_mask_count

0  Using MTCNN
Proper Mask: 0, No Mask: 1
1  Using MTCNN
Proper Mask: 1, No Mask: 0
2  Using MTCNN
Proper Mask: 2, No Mask: 0
3  Using MTCNN
Proper Mask: 3, No Mask: 0
4  Using Haarcascade
Proper Mask: 1, No Mask: 0
5  Using Haarcascade
Proper Mask: 2, No Mask: 0
6  Using MTCNN
Proper Mask: 9, No Mask: 0
7  Using Haarcascade
Proper Mask: 2, No Mask: 0
8  Using MTCNN
Proper Mask: 3, No Mask: 0
9  Using MTCNN
Proper Mask: 1, No Mask: 0
10  Using Haarcascade
Proper Mask: 1, No Mask: 0
11  Using Haarcascade
Proper Mask: 1, No Mask: 0
12  Using Haarcascade
Proper Mask: 1, No Mask: 0
13  Using MTCNN
Proper Mask: 0, No Mask: 1
14  Using MTCNN
Proper Mask: 2, No Mask: 0
15  Using MTCNN
Proper Mask: 1, No Mask: 0
16  Using MTCNN
Proper Mask: 0, No Mask: 0
17  Using MTCNN
Proper Mask: 4, No Mask: 0
18  Using MTCNN
Proper Mask: 1, No Mask: 0
19  Using MTCNN
Proper Mask: 0, No Mask: 0
20  Using MTCNN
Proper Mask: 1, No Mask: 0
21  Using MTCNN
Proper Mask: 1, No Mask: 0
22  Using MTCNN
Proper Mask:

186  Using MTCNN
Proper Mask: 4, No Mask: 0
187  Using MTCNN
Proper Mask: 1, No Mask: 0
188  Using MTCNN
Proper Mask: 3, No Mask: 0
189  Using MTCNN
Proper Mask: 1, No Mask: 0
190  Using Haarcascade
Proper Mask: 1, No Mask: 0
191  Using MTCNN
Proper Mask: 1, No Mask: 0
192  Using MTCNN
Proper Mask: 1, No Mask: 0
193  Using MTCNN
Proper Mask: 9, No Mask: 0
194  Using Haarcascade
Proper Mask: 2, No Mask: 0
195  Using Haarcascade
Proper Mask: 2, No Mask: 0
196  Using MTCNN
Proper Mask: 1, No Mask: 0
197  Using MTCNN
Proper Mask: 1, No Mask: 0
198  Using MTCNN
Proper Mask: 1, No Mask: 0
199  Using MTCNN
Proper Mask: 1, No Mask: 0
200  Using MTCNN
Proper Mask: 1, No Mask: 0
201  Using MTCNN
Proper Mask: 1, No Mask: 0
202  Using MTCNN
Proper Mask: 1, No Mask: 0
203  Using MTCNN
Proper Mask: 1, No Mask: 0
204  Using MTCNN
Proper Mask: 2, No Mask: 0
205  Using MTCNN
Proper Mask: 1, No Mask: 0
206  Using MTCNN
Proper Mask: 0, No Mask: 1
207  Using MTCNN
Proper Mask: 2, No Mask: 0
208  Using MTC

In [10]:
face_data

,images,mask_on,no_mask,num_people_image,pred_num_people,corr_pred,detect_type,pred_mask_on,pred_no_mask
0,mask-dataset/images/test/2815.png,0,1,1,1,1,MTCNN,NaN,NaN
1,mask-dataset/images/test/6436.jpg,1,0,1,1,1,MTCNN,NaN,NaN
2,mask-dataset/images/test/4035.jpg,1,1,2,2,1,MTCNN,NaN,NaN
3,mask-dataset/images/test/5848.jpg,3,0,3,3,1,MTCNN,NaN,NaN
4,mask-dataset/images/test/5603.jpg,2,0,2,2,1,Haarcascade,NaN,NaN
...,...,...,...,...,...,...,...,...,...
245,mask-dataset/images/test/3067.jpg,0,1,1,1,1,MTCNN,NaN,NaN
246,mask-dataset/images/test/3805.jpg,1,0,1,1,1,MTCNN,NaN,NaN
247,mask-dataset/images/test/4051.jpg,1,0,1,2,0,Haarcascade,NaN,NaN
248,mask-dataset/images/test/662.png,1,0,1,1,1,MTCNN,NaN,NaN


In [11]:
face_data.pred_mask_on.update(pd.Series(pred_mask_count))
face_data.pred_no_mask.update(pd.Series(pred_no_mask_count))

In [12]:
face_data['pred_mask_on'] = face_data['pred_mask_on'].astype(int)
face_data['pred_no_mask'] = face_data['pred_no_mask'].astype(int)

In [13]:
face_data.head(20)

,images,mask_on,no_mask,num_people_image,pred_num_people,corr_pred,detect_type,pred_mask_on,pred_no_mask
0,mask-dataset/images/test/2815.png,0,1,1,1,1,MTCNN,0,1
1,mask-dataset/images/test/6436.jpg,1,0,1,1,1,MTCNN,1,0
2,mask-dataset/images/test/4035.jpg,1,1,2,2,1,MTCNN,2,0
3,mask-dataset/images/test/5848.jpg,3,0,3,3,1,MTCNN,3,0
4,mask-dataset/images/test/5603.jpg,2,0,2,2,1,Haarcascade,1,0
5,mask-dataset/images/test/187.jpg,5,0,5,4,0,Haarcascade,2,0
6,mask-dataset/images/test/155.jpg,6,1,7,9,0,MTCNN,9,0
7,mask-dataset/images/test/2175.jpg,1,0,1,3,0,Haarcascade,2,0
8,mask-dataset/images/test/5065.jpg,3,0,3,3,1,MTCNN,3,0
9,mask-dataset/images/test/2652.png,1,0,1,1,1,MTCNN,1,0


In [14]:
face_data['corr_mask_pred'] = face_data.apply(
    lambda x: 1 if x['mask_on'] == x['pred_mask_on'] else 0, axis=1)

face_data['corr_no_mask_pred'] = face_data.apply(
    lambda x: 1 if x['no_mask'] == x['pred_no_mask'] else 0, axis=1)

In [15]:
face_data

,images,mask_on,no_mask,num_people_image,pred_num_people,corr_pred,detect_type,pred_mask_on,pred_no_mask,corr_mask_pred,corr_no_mask_pred
0,mask-dataset/images/test/2815.png,0,1,1,1,1,MTCNN,0,1,1,1
1,mask-dataset/images/test/6436.jpg,1,0,1,1,1,MTCNN,1,0,1,1
2,mask-dataset/images/test/4035.jpg,1,1,2,2,1,MTCNN,2,0,0,0
3,mask-dataset/images/test/5848.jpg,3,0,3,3,1,MTCNN,3,0,1,1
4,mask-dataset/images/test/5603.jpg,2,0,2,2,1,Haarcascade,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
245,mask-dataset/images/test/3067.jpg,0,1,1,1,1,MTCNN,0,1,1,1
246,mask-dataset/images/test/3805.jpg,1,0,1,1,1,MTCNN,1,0,1,1
247,mask-dataset/images/test/4051.jpg,1,0,1,2,0,Haarcascade,1,0,1,1
248,mask-dataset/images/test/662.png,1,0,1,1,1,MTCNN,1,0,1,1


In [19]:
mask_on_acc = sum(face_data['corr_mask_pred'])/len(face_data['corr_mask_pred'])
print(f'Overall mask on prediction accuracy = {mask_on_acc}')

Overall mask on prediction accuracy = 0.548


In [20]:
no_mask_acc = sum(face_data['corr_no_mask_pred'])/len(face_data['corr_no_mask_pred'])
print(f'Overall no mask prediction accuracy = {no_mask_acc}')

Overall no mask prediction accuracy = 0.712


### IMPROPER MASK ADDED TO NO MASK

In [21]:
face_data2 = pd.read_csv('updated_test_data.csv')
face_data2

,images,mask_on,no_mask,num_people_image,pred_num_people,corr_pred,detect_type
0,mask-dataset/images/test/2815.png,0,1,1,1,1,MTCNN
1,mask-dataset/images/test/6436.jpg,1,0,1,1,1,MTCNN
2,mask-dataset/images/test/4035.jpg,1,1,2,2,1,MTCNN
3,mask-dataset/images/test/5848.jpg,3,0,3,3,1,MTCNN
4,mask-dataset/images/test/5603.jpg,2,0,2,2,1,Haarcascade
...,...,...,...,...,...,...,...
245,mask-dataset/images/test/3067.jpg,0,1,1,1,1,MTCNN
246,mask-dataset/images/test/3805.jpg,1,0,1,1,1,MTCNN
247,mask-dataset/images/test/4051.jpg,1,0,1,2,0,Haarcascade
248,mask-dataset/images/test/662.png,1,0,1,1,1,MTCNN


In [23]:
face_data2['pred_mask_on'] = np.nan
face_data2['pred_no_mask'] = np.nan

In [24]:
pred_mask_count2 = {}
pred_no_mask_count2 = {}

for i in range(250):
    pic = face_data2.iloc[i]['images']
    detect = face_data2.iloc[i]['detect_type']
    im = Image.open(os.path.abspath(
        os.path.abspath(os.getcwd())+'/'+pic), mode='r')
    if im.mode != 'RGB':
        pixels = np.array(Image.open(os.path.abspath(
            os.getcwd())+'/'+pic, mode='r').convert('RGB'))
    elif im.mode == 'RGB':
        pixels = np.array(Image.open(
            os.path.abspath(os.getcwd())+'/'+pic, mode='r'))
    gray_image = cv2.cvtColor(pixels, cv2.COLOR_BGR2GRAY)

    im = cv2.imread(os.path.abspath(os.getcwd())+'/'+pic)
    
    #Haarcascades
    face_cascade = cv2.CascadeClassifier(os.path.abspath(os.getcwd())+'/haarcascade/haarcascade_frontalface_default.xml')
    profile_cascade = cv2.CascadeClassifier(os.path.abspath(os.getcwd())+'/haarcascade/haarcascade_profileface.xml')
    nose_cascade = cv2.CascadeClassifier(os.path.abspath(os.getcwd())+'/haarcascade/haarcascade_nose.xml')
    mouth_cascade = cv2.CascadeClassifier(os.path.abspath(os.getcwd())+'/haarcascade/haarcascade_mouth.xml')
    
    if detect == 'MTCNN':
        print(i, ' Using MTCNN')
        detector = MTCNN()

        # detect faces in the image
        faces = detector.detect_faces(pixels)

        # mask wear
        proper_mask_count = 0
#         improper_mask_count = 0
        no_mask_count = 0
        for result in faces:
            if result['confidence'] >= 0.75:
                x, y, w, h = result['box']
                cv2.rectangle(pixels, (x, y), (x+w, y+h), (255, 0, 0), 2)
                
                roi_image = gray_image[y:y+h, x:x+w]
                roi_color = pixels[y:y+h, x:x+w]
#                 plt.imshow(roi_color)
#                 ax = plt.gca()
                
#                 rect = Rectangle((x, y), w, h, fill=False, color='red')
# #             # draw the box
#                 ax.add_patch(rect)
                
                noses = nose_cascade.detectMultiScale(roi_image, scaleFactor=1.1, minNeighbors=20)
                for (nx,ny,nw,nh) in noses: 
                    cv2.rectangle(roi_image,(nx,ny),(nx+nw,ny+nh),(0,255,0),2)
#                     no = Rectangle((nx, ny), nw, nh, fill=False, color='green')
#                     ax.add_patch(no)
                mouth = mouth_cascade.detectMultiScale(roi_image, scaleFactor=1.3, minNeighbors=40)
                for (mx,my,mw,mh) in mouth: 
                    cv2.rectangle(roi_image,(mx,my),(mx+mw,my+mh),(0,0,255),2)
#                     mo = Rectangle((mx, my), mw, mh, fill=False, color='blue')
#                     ax.add_patch(mo)
#                 plt.show()

                num_nose = len(noses)
                num_mouth = len(mouth)
#                 print(f'Noses: {num_nose}, Mouths: {num_mouth}')
                
                if num_nose == 0 and num_mouth == 0: # Proper masking
                    proper_mask_count += 1
                elif num_nose != 0 and num_mouth == 0: # Improper masking
                    no_mask_count += 1
                elif num_nose != 0 and num_mouth != 0: # no masking
                    no_mask_count += 1
        print(f"Proper Mask: {proper_mask_count}, No Mask: {no_mask_count}")
                        
    if detect == "Haarcascade":
        print(i, ' Using Haarcascade')
        proper_mask_count = 0
#         improper_mask_count = 0
        no_mask_count = 0
        f = []
        detected_faces = face_cascade.detectMultiScale(image=im, scaleFactor=1.05, minNeighbors=8)
        detected_prof = profile_cascade.detectMultiScale(image=im, scaleFactor=1.05, minNeighbors=4)
        f.append(detected_faces)
        f.append(detected_prof)
        faces = [x for x in f if len(x) != 0]
        for face in faces:
            for (x, y, w, h) in face:
#                 print(x,y,w,h)
#                 cv2.rectangle(pixels, (x, y), (x+w, y+h), (255, 0, 0), 2)

                roi_image = gray_image[y:y+h, x:x+w]
                roi_color = pixels[y:y+h, x:x+w]
#                 plt.imshow(roi_color)
#                 ax = plt.gca()

#                 rect = Rectangle((x, y), w, h, fill=False, color='red')
#     #             # draw the box
#                 ax.add_patch(rect)

                noses = nose_cascade.detectMultiScale(roi_image, scaleFactor=1.1, minNeighbors=20)
                for (nx,ny,nw,nh) in noses: 
                    cv2.rectangle(roi_image,(nx,ny),(nx+nw,ny+nh),(0,255,0),2)
#                     no = Rectangle((nx, ny), nw, nh, fill=False, color='green')
#                     ax.add_patch(no)
                
                mouth = mouth_cascade.detectMultiScale(roi_image, scaleFactor=1.3, minNeighbors=40)
                for (mx,my,mw,mh) in mouth: 
                    cv2.rectangle(roi_image,(mx,my),(mx+mw,my+mh),(0,0,255),2)
#                     mo = Rectangle((mx, my), mw, mh, fill=False, color='blue')
#                     ax.add_patch(mo)
#                 plt.show()

            num_nose = len(noses)
            num_mouth = len(mouth)
#             print(f'Noses: {num_nose}, Mouths: {num_mouth}')

            if num_nose == 0 and num_mouth == 0: # Proper masking
                proper_mask_count += 1
            elif num_nose != 0 and num_mouth == 0: # Improper masking
                no_mask_count += 1
            elif num_nose != 0 and num_mouth != 0: # no masking
                no_mask_count += 1
        print(f"Proper Mask: {proper_mask_count}, No Mask: {no_mask_count}")

    pred_mask_count2[i] = proper_mask_count
    pred_no_mask_count2[i] = no_mask_count

0  Using MTCNN
Proper Mask: 0, No Mask: 1
1  Using MTCNN
Proper Mask: 1, No Mask: 0
2  Using MTCNN
Proper Mask: 2, No Mask: 0
3  Using MTCNN
Proper Mask: 3, No Mask: 0
4  Using Haarcascade
Proper Mask: 1, No Mask: 0
5  Using Haarcascade
Proper Mask: 2, No Mask: 0
6  Using MTCNN
Proper Mask: 8, No Mask: 1
7  Using Haarcascade
Proper Mask: 2, No Mask: 0
8  Using MTCNN
Proper Mask: 3, No Mask: 0
9  Using MTCNN
Proper Mask: 1, No Mask: 0
10  Using Haarcascade
Proper Mask: 1, No Mask: 0
11  Using Haarcascade
Proper Mask: 0, No Mask: 1
12  Using Haarcascade
Proper Mask: 1, No Mask: 0
13  Using MTCNN
Proper Mask: 0, No Mask: 1
14  Using MTCNN
Proper Mask: 2, No Mask: 0
15  Using MTCNN
Proper Mask: 1, No Mask: 0
16  Using MTCNN
Proper Mask: 0, No Mask: 0
17  Using MTCNN
Proper Mask: 4, No Mask: 0
18  Using MTCNN
Proper Mask: 1, No Mask: 0
19  Using MTCNN
Proper Mask: 0, No Mask: 0
20  Using MTCNN
Proper Mask: 1, No Mask: 0
21  Using MTCNN
Proper Mask: 1, No Mask: 0
22  Using MTCNN
Proper Mask:

Proper Mask: 3, No Mask: 1
187  Using MTCNN
Proper Mask: 1, No Mask: 0
188  Using MTCNN
Proper Mask: 3, No Mask: 0
189  Using MTCNN
Proper Mask: 1, No Mask: 0
190  Using Haarcascade
Proper Mask: 1, No Mask: 0
191  Using MTCNN
Proper Mask: 1, No Mask: 0
192  Using MTCNN
Proper Mask: 1, No Mask: 0
193  Using MTCNN
Proper Mask: 5, No Mask: 4
194  Using Haarcascade
Proper Mask: 2, No Mask: 0
195  Using Haarcascade
Proper Mask: 2, No Mask: 0
196  Using MTCNN
Proper Mask: 1, No Mask: 0
197  Using MTCNN
Proper Mask: 1, No Mask: 0
198  Using MTCNN
Proper Mask: 0, No Mask: 1
199  Using MTCNN
Proper Mask: 0, No Mask: 1
200  Using MTCNN
Proper Mask: 1, No Mask: 0
201  Using MTCNN
Proper Mask: 1, No Mask: 0
202  Using MTCNN
Proper Mask: 1, No Mask: 0
203  Using MTCNN
Proper Mask: 1, No Mask: 0
204  Using MTCNN
Proper Mask: 2, No Mask: 0
205  Using MTCNN
Proper Mask: 1, No Mask: 0
206  Using MTCNN
Proper Mask: 0, No Mask: 1
207  Using MTCNN
Proper Mask: 2, No Mask: 0
208  Using MTCNN
Proper Mask: 2

In [25]:
face_data2.pred_mask_on.update(pd.Series(pred_mask_count2))
face_data2.pred_no_mask.update(pd.Series(pred_no_mask_count2))

In [26]:
face_data2['pred_mask_on'] = face_data2['pred_mask_on'].astype(int)
face_data2['pred_no_mask'] = face_data2['pred_no_mask'].astype(int)

In [27]:
face_data2['corr_mask_pred'] = face_data2.apply(
    lambda x: 1 if x['mask_on'] == x['pred_mask_on'] else 0, axis=1)

face_data2['corr_no_mask_pred'] = face_data2.apply(
    lambda x: 1 if x['no_mask'] == x['pred_no_mask'] else 0, axis=1)

In [28]:
face_data2.head(10)

,images,mask_on,no_mask,num_people_image,pred_num_people,corr_pred,detect_type,pred_mask_on,pred_no_mask,corr_mask_pred,corr_no_mask_pred
0,mask-dataset/images/test/2815.png,0,1,1,1,1,MTCNN,0,1,1,1
1,mask-dataset/images/test/6436.jpg,1,0,1,1,1,MTCNN,1,0,1,1
2,mask-dataset/images/test/4035.jpg,1,1,2,2,1,MTCNN,2,0,0,0
3,mask-dataset/images/test/5848.jpg,3,0,3,3,1,MTCNN,3,0,1,1
4,mask-dataset/images/test/5603.jpg,2,0,2,2,1,Haarcascade,1,0,0,1
5,mask-dataset/images/test/187.jpg,5,0,5,4,0,Haarcascade,2,0,0,1
6,mask-dataset/images/test/155.jpg,6,1,7,9,0,MTCNN,8,1,0,1
7,mask-dataset/images/test/2175.jpg,1,0,1,3,0,Haarcascade,2,0,0,1
8,mask-dataset/images/test/5065.jpg,3,0,3,3,1,MTCNN,3,0,1,1
9,mask-dataset/images/test/2652.png,1,0,1,1,1,MTCNN,1,0,1,1


In [29]:
mask_on_acc2 = sum(face_data2['corr_mask_pred'])/len(face_data2['corr_mask_pred'])
print(f'Overall mask on prediction accuracy = {mask_on_acc2}')

Overall mask on prediction accuracy = 0.6


In [30]:
no_mask_acc2 = sum(face_data2['corr_no_mask_pred'])/len(face_data2['corr_no_mask_pred'])
print(f'Overall no mask prediction accuracy = {no_mask_acc2}')

Overall no mask prediction accuracy = 0.768
